# Cleanup, TODO and References

---

## Cleanup

The cleanup procedure for the AWS resources follows closely the Amazon Forecast Tutorial here:

https://github.com/aws-samples/amazon-forecast-samples/blob/master/notebooks/basic/Tutorial/4.Cleanup.ipynb


## TODO:

Multivariate
compare metrics MAE, RMSE, MAPE, WAPE


## References



In [ ]:

import sys
import os
import json
import time

import boto3
import pandas as pd

import util

In [ ]:
# Retrieve  shared variables from the earlier notebooks.
%store -r

In [ ]:
# %store -r

# # Print your choices from first notebook
# print(f"item_id = {item_id}")
# print(f"project = {PROJECT}")
# print(f"data_version = {DATA_VERSION}")
# print(f"Forecast length = {FORECAST_LENGTH}")
# print(f"Dataset frequency = {DATASET_FREQUENCY}")
# print(f"Timestamp format = {TIMESTAMP_FORMAT}")
# print(f"dataset_group_arn = {dataset_group_arn}")
# print(f"role_arn = {role_arn}")
# %store -r bucket_name
# print(f"bucket_name = {bucket_name}")
# %store -r region
# print(f"region = {region}")

In [ ]:
# connect to the Forecast APIs via the SDK.

ession = boto3.Session(region_name= region) 
forecast = session.client(service_name= 'forecast') 
forecastquery = session.client(service_name= 'forecastquery')

In [ ]:

# Define the Things to Cleanup
# In the previous notebooks several variables were stored at the end of each.  Now that they have been retrieved above, the cells below will delete the items that were created one at a time until all items that were created have been removed. 
# For a parent resource, all its child resources can be deleted using `delete_resource_tree` -- see AWS DeleteResourceTree API:
# https://docs.aws.amazon.com/forecast/latest/dg/API_DeleteResourceTree.html
# Bellow cell uses `delete_resource_tree` to delete the predictor resource and all its child resources such as Forecasts, PredictorBacktestExportJobs and ForecastExportJobs.

In [ ]:
# Delete predictor and all its child resources such as Forecasts, PredictorBacktestExportJobs and ForecastExportJobs 
util.wait_till_delete(lambda: forecast.delete_resource_tree(ResourceArn = predictor_arn_deep_ar))

In [ ]:
# Delete the target time series dataset import job
util.wait_till_delete(lambda: forecast.delete_dataset_import_job(DatasetImportJobArn=ts_dataset_import_job_arn))

In [ ]:
# Delete the target time series dataset
util.wait_till_delete(lambda: forecast.delete_dataset(DatasetArn=ts_dataset_arn))

In [ ]:
# Delete dataset group
util.wait_till_delete(lambda: forecast.delete_dataset_group(DatasetGroupArn=dataset_group_arn))

In [ ]:
# Delete the files in S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).delete()

In [ ]:
# IAM Role and Policy Cleanup
# Remove the policies that were attached to a role and then to delete it. 


util.delete_iam_role( role_arn )


## TODO:


## References


1. Magnus Erik Hvass Pedersen, [TensorFlow-Tutorials](http://www.hvass-labs.org/)
/ [GitHub repo](https://github.com/Hvass-Labs/TensorFlow-Tutorials) / [Videos on YouTube](https://www.youtube.com/playlist?list=PL9Hr9sNUjfsmEu1ZniY0XpHSzl5uihcXZ)

2. Ognian Dantchev, Multivariate Time Series Forecasting with Keras and TensorFlow, [GitHub repo](https://github.com/ogniandantchev/dilated_causal_cnn_time_series)

3. Amazon Forecast resources, [AWS website](https://aws.amazon.com/forecast/resources/)

4. Time Series Forecasting Principles with Amazon Forecast, [Technical Guide](https://d1.awsstatic.com/whitepapers/time-series-forecasting-principles-amazon-forecast.pdf)

5. AWS Samples-- Amazon Forecast Samples [GitHub repo](https://github.com/aws-samples/amazon-forecast-samples)


